In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

from pathta import Study
from frites import set_mpl_style
from frites.conn import conn_links
from research.study.pblt import get_model, get_anat_table
from visbrain.utils import normalize

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
import seaborn as sns

from ipywidgets import interact

import json
with open("../config.json", 'r') as f:
    cfg = json.load(f)

set_mpl_style('frites')

plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 25
plt.rcParams['axes.labelsize'] = 24

# **I/O**
---

In [ ]:
st = Study('PBLT')
subjects = list(st.load_config('subjects.json').keys())
rp_ref = {
    '+0€': 'Rewarding', '+1€': 'Rewarding',
    '-0€': 'Punishing', '-1€': 'Punishing'
}

# load all of the behavioral tables
beh = []
for n_s, s in enumerate(subjects):
    # load the subject specific behavioral table
    _df = st.search(s, folder='behavior', load=True).rename(
        columns={'outcome valence': 'outcome'})
    _df['subject'] = s
    _df['subject #'] = n_s + 1
    
    # get the condition
    _df['condition'] = _df['outcome'].replace(rp_ref)

    # load the PE
    _df['PE'] = get_model('PE', s)
    beh.append(_df[[
        'condition', 'PE', 'subject', 'subject #', 'outcome', 'trial index',
        'pair of stimuli'
    ]])
beh = pd.concat(beh).reset_index(drop=True)
beh["PEName"] = [{"Rewarding": "RPE", "Punishing": "PPE"}[p] for p in beh['condition']]
beh.rename(columns={"trial index": "Trials"}, inplace=True)

# **Plot**
---
## Lineplot with ci

In [ ]:
fg = sns.FacetGrid(beh, col='subject #', hue='PEName', col_wrap=6, palette=['C1', 'C0'])
fg.map_dataframe(sns.lineplot, x='Trials', y='PE')
fg.add_legend(title="")
sns.move_legend(fg, "lower right", bbox_to_anchor=(.9, 0.15), frameon=True)
for s in range(17):
    plt.sca(fg.axes[s])
    plt.title(f"Subject {s + 1}")

In [ ]:
save_to = cfg['export']['review']
cfg_export = cfg['export']['cfg']

fg.figure.savefig(f'{save_to}/PE_single_subject.png', **cfg_export)

## Lineplot without ci

In [ ]:
fig, axs = plt.subplots(3, 6, figsize=(22, 12), sharex=True, sharey=True)
axs = np.ravel(axs)

for n_s in range(17):
    plt.sca(axs[n_s])
    for n_c, c in enumerate(['Rewarding', 'Punishing']):
        _beh = beh.set_index(['subject #', 'condition']).loc[n_s + 1].loc[c]

        palette = "Reds" if c == 'Punishing' else "Blues"

        sns.lineplot(
            data=_beh, x="Trials", y="PE", hue="pair of stimuli", palette=palette,
            legend=False, alpha=.8
        )
    plt.title("Subject %s" % (n_s + 1))